In [1]:
from torch import empty, zeros
import math

In [2]:
class Module(object):
    def __init__(self):
        pass
        
    def forward(self, *input_):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []
    
    def update(self, lr):
        pass
    
    def zero_grad(self):
        pass


In [15]:
class Linear(Module):
    def __init__(self, N_in, N_out):
        super(Linear, self).__init__()
        self.N_in = N_in
        self.N_out = N_out
        
        self.W = empty((N_in, N_out)).normal_()
        self.b = empty((1, N_out)).normal_()
        
        self.gradW = zeros(self.W.shape).normal_()
        self.gradb = zeros(self.b.shape).normal_()
        
    def forward(self, *input_):
        # out = W * input + b
        x = input_[0].clone()
        
        self.x = x
        
        return self.x.mm(self.W) + self.b
        
    def backward(self, *gradwrtoutput):
        # grad_w += input * x^(l-1).t()
        # grad_b += input
        # out = w.t() * input
        # input = grad of activation function, i.e. dl/ds^(l)
        # x^(l-1) = input of the forward pass
        input_ = gradwrtoutput[0].clone()
        
        self.gradW += self.x.t().mm(input_)
        self.gradb += input_.sum(0)
        
        return input_.mm(self.W.t())
        
    def param(self):
        return [(self.W, self.gradW), (self.b, self.gradb)]
    
    def update(self, lr):
        self.W.sub_(lr * self.gradW)
        self.b.sub_(lr * self.gradb)
    
    def zero_grad(self):
        self.gradW = zeros(self.W.shape)
        self.gradb = zeros(self.b.shape)

In [4]:
class ReLU(Module):
    def __init__(self):
        super(ReLU, self).__init__()
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        s[s < 0] = 0.
        
        return s
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out[out > 0] = 1
        out[out < 0] = 0
        
        
        return out.mul(input_)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

In [5]:
class Tanh(Module):
    def __init__(self):
        super(Tanh, self).__init__()
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        return s.tanh()
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out = 1 - out.tanh().pow(2)
        
        return out.mul(input_)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

In [6]:
class Sequential(Module):
    def __init__(self, *modules):
        super(Sequential, self).__init__()
        self.modules = modules
        
    def forward(self, *input_):
        x = input_[0].clone()
        
        for m in self.modules:
            x = m.forward(x)
        
        return x
        
    def backward(self, *gradwrtoutput):
        x = gradwrtoutput[0].clone()
        
        for i, m in enumerate(reversed(self.modules)):
            #print("{} : {}".format(i, x))
            x = m.backward(x)
        
    def param(self):
        params = []
        
        for m in self.modules:
            for param in m.param():
                params.append(param)
        
        return params

    def update(self, lr):
        for m in self.modules:
            m.update(lr)
    
    def zero_grad(self):
        for m in self.modules:
            m.zero_grad()

In [7]:
"""class Loss: 
    ...
"""
class LossMSE(Module):
    def __init__(self):
        super(LossMSE, self).__init__()
        pass
        
    def forward(self, y, target):
        # out = e^2
        # e = (y - f(x))
        self.y = y.clone()
        target_onehot = zeros((target.shape[0], 2)) 
        self.target = target_onehot.scatter_(1, target.view(-1, 1), 1)
        
        self.e = (self.y - self.target)
        
        
        return self.e.pow(2).sum()
        
    def backward(self):
        # out = 2 * e
        
        return 2 * self.e
        
        

In [8]:
class Optimizer:
    def __init__(self):
        pass
    
    def step(self):
        raise NotImplementedError

class SGD(Optimizer):
    def __init__(self, model, lr):
        super(SGD, self).__init__()
        self.model = model
        self.lr = lr
    
    # Train function?
    # def train(..):
    
    def step(self):
        model.update(self.lr)

In [9]:
def generate_disc_set(nb):
    input = empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

In [10]:
train_input

tensor([[ 0.4521,  0.0235],
        [-0.0999,  0.8495],
        [-0.4522, -0.3818],
        ...,
        [-0.6159, -0.8177],
        [-0.2290,  0.1896],
        [-0.9179,  0.2292]])

In [21]:
def train_model(model, train_input, train_target, mini_batch_size=1):
    criterion = LossMSE()
    optimizer = SGD(model, lr = 1e-4)
    nb_epochs = 50

    for e in range(nb_epochs):
        sum_loss = 0.
        
        for b in range(0, train_input.size(0), mini_batch_size):
            model.zero_grad()
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            #print(output)
            loss = criterion.forward(output, train_target.narrow(0, b, mini_batch_size))
            
            sum_loss += loss
            
            l_grad = criterion.backward()
            model.backward(l_grad)
            #print(model.param()[0])
            optimizer.step()
            
        #print(model.param()[0])
            
        print("{} iteration: loss={}".format(e, sum_loss))

In [25]:
model = Sequential(Linear(2, 25), ReLU(),
                   Linear(25, 25), ReLU(),
                   Linear(25, 25), ReLU(),
                   Linear(25, 2))


train_model(model, train_input, train_target, 4)

0 iteration: loss=103039.765625
1 iteration: loss=1267.904541015625
2 iteration: loss=942.134521484375
3 iteration: loss=793.8841552734375
4 iteration: loss=696.9398803710938
5 iteration: loss=624.7383422851562
6 iteration: loss=568.2059936523438
7 iteration: loss=523.1644287109375
8 iteration: loss=486.41412353515625
9 iteration: loss=454.4186706542969
10 iteration: loss=427.0074157714844
11 iteration: loss=404.56329345703125
12 iteration: loss=384.3256530761719
13 iteration: loss=365.3061218261719
14 iteration: loss=349.16021728515625
15 iteration: loss=335.129150390625
16 iteration: loss=322.6468811035156
17 iteration: loss=311.48321533203125
18 iteration: loss=301.76947021484375
19 iteration: loss=291.84222412109375
20 iteration: loss=284.13641357421875
21 iteration: loss=276.53497314453125
22 iteration: loss=269.75921630859375
23 iteration: loss=263.9503173828125
24 iteration: loss=258.34381103515625
25 iteration: loss=254.33367919921875
26 iteration: loss=249.36134338378906
27 it

In [26]:
import torch
def test(model, test_input, test_target):
    num_samples = test_input.size(0)
    prediction = model.forward(test_input)
    predicted_class = torch.argmax(prediction, axis=1)
    accuracy = sum(predicted_class == test_target).float() / num_samples
    return accuracy

In [27]:
test(model, test_input, test_target)

tensor(0.9220)